In [ ]:
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from scipy.fft import fft
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/project

/content/drive/MyDrive/project


In [ ]:
L=8064
Fs=128
Features=np.zeros((25600,32,3*128))
Labels=list()
for subject_id in range(0,32):
    if subject_id % 10 ==0:
      print(subject_id)
    loaded_data=None
    file_name=None
    if subject_id<9:
        file_name= 'data_preprocessed_python/s0'+str(subject_id+1)+'.dat'
    else:
        file_name= 'data_preprocessed_python/s'+str(subject_id+1)+'.dat'
    with open(file_name, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        loaded_data = u.load()
    data=loaded_data['data']
    Labels.append(loaded_data['labels'])
    
    for i in range(len(data)):
        for k in range(3,61,3):
          for j in range(32):   
            x=np.array(data[i][j][k*128:(k+3)*128]) -np.array((data[i][j][:3*128]))
            x=zscore(x)
            Features[subject_id*800+i*20+int(k/3-1)][j]=x

0
10
20
30


In [ ]:
def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))   

def ShallowConvNet(nb_classes, Chans = 64, Samples = 128, dropoutRate = 0.5):
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(40, (1, 13), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(40, (Chans, 1), use_bias=False, 
                          kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block1)
    block1       = Activation(square)(block1)
    block1       = AveragePooling2D(pool_size=(1, 35), strides=(1, 7))(block1)
    block1       = Activation(log)(block1)
    block1       = Dropout(dropoutRate)(block1)
    flatten      = Flatten()(block1)
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)

In [ ]:
y=np.zeros(800*32)
for j in range(32):
  for i in range(40):
    if Labels[j][i][1]>5:
      y[j*800+i*20:j*800+(i+1)*20]=1

In [ ]:
def run(train_index,test_index,
        dropout_rate=0.5):



  Y=np_utils.to_categorical(y)

  d_size, chans, samples = Features.shape
  
  model=ShallowConvNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = dropout_rate)

  # compile the model and set the optimizers
  model.compile(loss='categorical_crossentropy', optimizer='adam', 
                metrics = ['accuracy'])
  
  # count number of parameters in the model
  checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                                save_best_only=True)
  
  
  class_weights = {0:np.sum(y)/(len(y)-np.sum(y)), 1:1}
  
  fittedModel = model.fit(Features[train_index], Y[train_index], batch_size = 64, epochs =30 , 
                          verbose=2, validation_data=(Features[test_index], Y[test_index]),
                          callbacks=[checkpointer],class_weight=class_weights)
  probs       = model.predict(Features[test_index])
  preds       = probs.argmax(axis = -1)
  test = Y[test_index].argmax(axis=-1)
  acc = accuracy_score(test, preds)
  cm = confusion_matrix(test, preds)
  f1= f1_score(test, preds)
  return test, preds, acc,f1 , cm, model
  

In [ ]:
kf=KFold(n_splits=5,shuffle=True,random_state=0)

In [ ]:
train_index=list(kf.split(Features))[0][0]
test_index=list(kf.split(Features))[0][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 47s - loss: 0.7366 - accuracy: 0.6586 - val_loss: 0.5273 - val_accuracy: 0.7451

Epoch 00001: val_loss improved from inf to 0.52729, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 17s - loss: 0.5536 - accuracy: 0.7748 - val_loss: 0.4240 - val_accuracy: 0.8100

Epoch 00002: val_loss improved from 0.52729 to 0.42396, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 17s - loss: 0.4665 - accuracy: 0.8268 - val_loss: 0.3579 - val_accuracy: 0.8672

Epoch 00003: val_loss improved from 0.42396 to 0.35793, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 17s - loss: 0.4216 - accuracy: 0.8515 - val_loss: 0.3382 - val_accuracy: 0.8689

Epoch 00004: val_loss improved from 0.35793 to 0.33820, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 17s - loss: 0.3993 - accuracy: 0.8646 - val_loss: 0.3468 - val_accuracy: 0.8643

Epoch 00005: val_loss did not improve from 0.33820
Epoch 6/30
320/320 - 17s - loss: 0.3758 - accuracy: 0.8758 - val_loss: 0.3369 -

In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 0, 0]),
 array([1, 1, 1, ..., 0, 0, 0]),
 0.91796875,
 0.9288858787673553,
 array([[1957,  186],
        [ 234, 2743]]),
 <keras.engine.functional.Functional at 0x7f9140652c10>)

In [ ]:
train_index=list(kf.split(Features))[1][0]
test_index=list(kf.split(Features))[1][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 18s - loss: 0.7202 - accuracy: 0.6725 - val_loss: 0.5004 - val_accuracy: 0.7600

Epoch 00001: val_loss improved from inf to 0.50042, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 17s - loss: 0.5477 - accuracy: 0.7802 - val_loss: 0.4077 - val_accuracy: 0.8307

Epoch 00002: val_loss improved from 0.50042 to 0.40770, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 17s - loss: 0.4644 - accuracy: 0.8264 - val_loss: 0.3666 - val_accuracy: 0.8584

Epoch 00003: val_loss improved from 0.40770 to 0.36656, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 17s - loss: 0.4209 - accuracy: 0.8487 - val_loss: 0.3393 - val_accuracy: 0.8723

Epoch 00004: val_loss improved from 0.36656 to 0.33929, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 17s - loss: 0.3977 - accuracy: 0.8678 - val_loss: 0.3278 - val_accuracy: 0.8773

Epoch 00005: val_loss improved from 0.33929 to 0.32778, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 17s - loss: 0.3740

In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 0, 0]),
 array([0, 1, 1, ..., 0, 0, 0]),
 0.9154296875,
 0.9235117470411588,
 array([[2073,  127],
        [ 306, 2614]]),
 <keras.engine.functional.Functional at 0x7f9140201410>)

In [ ]:
train_index=list(kf.split(Features))[2][0]
test_index=list(kf.split(Features))[2][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 19s - loss: 0.7269 - accuracy: 0.6713 - val_loss: 0.5044 - val_accuracy: 0.7580

Epoch 00001: val_loss improved from inf to 0.50439, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 17s - loss: 0.5438 - accuracy: 0.7790 - val_loss: 0.4458 - val_accuracy: 0.7963

Epoch 00002: val_loss improved from 0.50439 to 0.44585, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 17s - loss: 0.4669 - accuracy: 0.8239 - val_loss: 0.3746 - val_accuracy: 0.8564

Epoch 00003: val_loss improved from 0.44585 to 0.37462, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 17s - loss: 0.4282 - accuracy: 0.8453 - val_loss: 0.3398 - val_accuracy: 0.8791

Epoch 00004: val_loss improved from 0.37462 to 0.33979, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 17s - loss: 0.4004 - accuracy: 0.8623 - val_loss: 0.3380 - val_accuracy: 0.8762

Epoch 00005: val_loss improved from 0.33979 to 0.33800, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 17s - loss: 0.3847

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 0, 0]),
 array([1, 1, 0, ..., 1, 0, 0]),
 0.887109375,
 0.8942940746159473,
 array([[2097,  118],
        [ 460, 2445]]),
 <keras.engine.functional.Functional at 0x7f91402a0590>)

In [ ]:
train_index=list(kf.split(Features))[3][0]
test_index=list(kf.split(Features))[3][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 18s - loss: 0.7349 - accuracy: 0.6597 - val_loss: 0.5102 - val_accuracy: 0.7594

Epoch 00001: val_loss improved from inf to 0.51018, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 17s - loss: 0.5520 - accuracy: 0.7739 - val_loss: 0.4221 - val_accuracy: 0.8174

Epoch 00002: val_loss improved from 0.51018 to 0.42209, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 17s - loss: 0.4693 - accuracy: 0.8193 - val_loss: 0.3929 - val_accuracy: 0.8291

Epoch 00003: val_loss improved from 0.42209 to 0.39294, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 17s - loss: 0.4225 - accuracy: 0.8504 - val_loss: 0.3534 - val_accuracy: 0.8615

Epoch 00004: val_loss improved from 0.39294 to 0.35335, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 17s - loss: 0.4013 - accuracy: 0.8606 - val_loss: 0.3398 - val_accuracy: 0.8664

Epoch 00005: val_loss improved from 0.35335 to 0.33981, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 17s - loss: 0.3773

In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 0, 0]),
 array([1, 1, 1, ..., 0, 0, 0]),
 0.9025390625,
 0.917040731504572,
 array([[1863,  251],
        [ 248, 2758]]),
 <keras.engine.functional.Functional at 0x7f9140052790>)

In [ ]:
train_index=list(kf.split(Features))[4][0]
test_index=list(kf.split(Features))[4][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 47s - loss: 0.7351 - accuracy: 0.6612 - val_loss: 0.5274 - val_accuracy: 0.7381

Epoch 00001: val_loss improved from inf to 0.52736, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 17s - loss: 0.5614 - accuracy: 0.7681 - val_loss: 0.4213 - val_accuracy: 0.8191

Epoch 00002: val_loss improved from 0.52736 to 0.42133, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 17s - loss: 0.4678 - accuracy: 0.8250 - val_loss: 0.3640 - val_accuracy: 0.8604

Epoch 00003: val_loss improved from 0.42133 to 0.36397, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 17s - loss: 0.4268 - accuracy: 0.8486 - val_loss: 0.3463 - val_accuracy: 0.8658

Epoch 00004: val_loss improved from 0.36397 to 0.34629, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 17s - loss: 0.3989 - accuracy: 0.8654 - val_loss: 0.3442 - val_accuracy: 0.8625

Epoch 00005: val_loss improved from 0.34629 to 0.34418, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 17s - loss: 0.3767

In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 0, 0]),
 array([1, 1, 1, ..., 1, 1, 0]),
 0.9255859375,
 0.9341855242701677,
 array([[2035,  153],
        [ 228, 2704]]),
 <keras.engine.functional.Functional at 0x7f804033d250>)

In [ ]:
results=[]
for train_index, test_index in kf.split(Features):
  res=run(train_index,test_index)
  results.append(res)
  print(res)

Epoch 1/30
320/320 - 18s - loss: 0.7240 - accuracy: 0.6426 - val_loss: 0.5208 - val_accuracy: 0.7592

Epoch 00001: val_loss improved from inf to 0.52084, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 17s - loss: 0.5542 - accuracy: 0.7589 - val_loss: 0.4520 - val_accuracy: 0.8010

Epoch 00002: val_loss improved from 0.52084 to 0.45199, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 17s - loss: 0.4749 - accuracy: 0.8104 - val_loss: 0.4008 - val_accuracy: 0.8449

Epoch 00003: val_loss improved from 0.45199 to 0.40076, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 17s - loss: 0.4250 - accuracy: 0.8417 - val_loss: 0.3654 - val_accuracy: 0.8625

Epoch 00004: val_loss improved from 0.40076 to 0.36542, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 16s - loss: 0.4006 - accuracy: 0.8557 - val_loss: 0.3770 - val_accuracy: 0.8490

Epoch 00005: val_loss did not improve from 0.36542
Epoch 6/30
320/320 - 17s - loss: 0.3849 - accuracy: 0.8673 - val_loss: 0.3460 -